In [283]:
import numpy as np

p = 3
K = GF(p)
n = 3
m = 2
# R = PolynomialRing(K, x, n) # defines n variables over K
# R.inject_variables(verbose=False) # makes all the variables ready for use

In [284]:
# class QuadraticForm(SageObject):
# adjusted Gram matrix method that works for any ring of char != 2
def my_Gram_matrix(self):
    R = self.base_ring()
    assert R.characteristic() != 2, "Characteristic of the base ring cannot be 2."
    A = (R(1) / R(2)) * self.matrix()
    n = self.dim()

    Int_flag = True
    for i in range(n):
        for j in range(i, n):
            Int_flag &= A[i, j] in R

    # Return the Gram matrix, or an error
    if Int_flag:
        return MatrixSpace(R, n, n)(A)
    raise TypeError("this form does not have an integral Gram matrix")

QuadraticForm.my_Gram_matrix = my_Gram_matrix

In [210]:
# given a polynomial f, output the matrix A, vector B and scalar C s.t.
# f(x) = x^T Ax + Bx + C
def quadratic_to_matrix(f):
    h = f.homogeneous_components()
    A = QuadraticForm(h.get(2)).my_Gram_matrix()
    B = vector([])
    C = 0

    # try/except but for the poor ppl 
    if h.get(1) != None:
        # we make a vector of the coefficients of each basis element
        B = vector([h.get(1).coefficient(R.gens()[i]) for i in range(n)])

    if h.get(0) != None:
        C = h.get(0)

    return A, B, C

In [ ]:
# a function that simulates precomposition of a quadratic form
# f(x) = x^T Ax + Bx + C with a n x n matrix S 
# so it returns f \circ S 
def transform_input(S, A, B, C):
    # this throws an error because S^T A S is a matrix but BS is a vector and you cannot
    # put them in together in one vector
    #
    # it works if you return them separately, not in a vector
    return S.transpose()*A*S, B*S, C

In [211]:
def random_symmetric_matrix(K, n):
    # get a random matrix
    A = random_matrix(K, n, n)
    # and make it symmetric
    return A + A.transpose()

# returns A, B and C like above
def random_quadratic_poly(K, n):
    return random_symmetric_matrix(K, n), random_vector(K, n), K.random_element()

In [323]:
class VectorOfMatrices:
    def __init__(self, matrices):
        self.matrices = matrices
        self.len = len(matrices)

    def get_len(self):
        return self.len
     
    def __mul__(self, other):
        # https://doc.sagemath.org/html/en/reference/matrices/sage/matrix/matrix_dense.html
        if not isinstance(other, sage.matrix.matrix_dense.Matrix_dense):
            raise TypeError("We only multiply by a matrix my dear")

        if other.ncols() != self.len:
            raise ValueError("Matrix dimensions do not match")

        # multiply them by hand
        result = [sum(other[i, j] * self.matrices[j] for j in range(self.len)) for i in range(other.nrows())]
        # gives back a list
        return result

    def __sub__(self, other):
        # https://doc.sagemath.org/html/en/reference/modules/sage/modules/vector_modn_dense.html#sage.modules.vector_modn_dense.Vector_modn_dense
        # if not isinstance(other, sage.modules.vector_modn_dense):
            # raise TypeError("We only subtract other VectorOfMatrices my dear")

        if other.len != self.len:
            raise ValueError("Vector dimensions do not match")

        result = [self.matrices[i] - other.matrices[i] for i in range(self.len)]
        return result
        
    def __repr__(self):
        return f"VectorOfMatrices(\n{self.matrices})"
    
    __rmul__ = __mul__
    __rsub__ = __sub__

In [324]:
# an actual example
# instead of generating an actual polynomial first 
# and then getting the corresponding matrices,
# generate the matrices randomly

# here are the actual transformations
T = matrix(GL(m, K).random_element())
S = matrix(GL(n, K).random_element())

# We define the ring with enough variables for both our matrices
R2  = PolynomialRing(K, n^2 + m^2, 'y')
R2.inject_variables(verbose=False)

# first n^2 elements of R2
S_ = matrix(R2, n, n, R2.gens()[:n^2])

# we get the last m^2 variables of R2
T_ = matrix(R2, m, m, R2.gens()[-m^2:])
# print(S_, T_)

# generate whole ass m-sized vector of (homogenous) polynomials
hom = []
vec = []
scal = []
for i in range(m):
    hom.append(random_symmetric_matrix(K, n))
    vec.append(random_vector(K, n))
    scal.append(K.random_element())

In [297]:
# this is for the homogenous part S^T AS
G = [S.transpose()*A*S for A in hom]
G_ = [S_.transpose()*A*S_ for A in hom]

v = VectorOfMatrices(G)
v_ = VectorOfMatrices(G_)

P  = [[],[],[]]
P_ = [[],[],[]]
P[0] = T*v
P_[0] = T_*v_

P[1] = T*matrix([B*S for B in vec])
P_[1] = T_*matrix([B*S_ for B in vec])

P[2] = T*vector(scal)
P_[2] = T_*vector(scal)

# add the field equations by hand
system = [y^p - y for y in R2.gens()]
for i in range(m):
    system += (P_[0][i] - P[0][i]).list()
    system += (P_[1][i] - P[1][i]).list()
    system += [P_[2][i] - P[2][i]]

# the list(set(.)) removes the duplicates
total_system = list(set(system))
# print(total_system)
# print(type(P[1][1]))
# print(P_[1][1])

In [342]:
# this is for the homogenous part S^T AS
G = [S.transpose()*A*S for A in hom]
G_ = [S_.transpose()*A*S_ for A in hom]

v = VectorOfMatrices(G)
v_ = VectorOfMatrices(G_)

P  = [[],[],[]]
P_ = [[],[],[]]
Q  = [[],[],[]]
P[0] = T*v
P_[0] = T_*VectorOfMatrices(P[0])
Q[0] = VectorOfMatrices(G_)

P[1] = T*matrix([B*S for B in vec])
P_[1] = T_*P[1]
Q[1] = matrix([B*S_ for B in vec])

P[2] = T*vector(scal)
P_[2] = T_*P[2]
Q[2] = vector(scal)

# add the field equations by hand
system = [y^p - y for y in R2.gens()]
# system += VectorOfMatrices(P_[0]) - Q[0]
M = VectorOfMatrices(P_[0]) - Q[0]
for i in range(m):
    system += M[i].list()
    system += (P_[1][i] - Q[1][i]).list()
    system += [P_[2][i] - Q[2][i]]

# the list(set(.)) removes the duplicates
total_system = list(set(system))
print(total_system)

# print(VectorOfMatrices(P[0]) - VectorOfMatrices(P_[0]))
# print(P_[0][0])
# print(type(P_[0]))
# print(P[1][1])
# print(P_[1][1])
# t = T_.inverse()
# print((t*P_[1])[1])

[y1^3 - y1, -y2^2 - y2*y5 + y5^2 + y9, y9 + y10 - 1, y2 + y5 - y10, y0^3 - y0, y2^3 - y2, -y0*y1 + y1*y3 + y0*y4 + y3*y4 + y9 + y10, -y2*y5 - y5^2 - y2*y8 - y5*y8 + y11, y11^3 - y11, y2*y4 + y1*y5 - y4*y5 + y2*y7 + y5*y7 + y1*y8 + y4*y8 - y11 - y12, y5^3 - y5, y6^3 - y6, -y0^2 - y0*y3 + y3^2 - y9, y2*y3 + y0*y5 - y3*y5 + y2*y6 + y5*y6 + y0*y8 + y3*y8 + y12, y9^3 - y9, y1*y3 + y0*y4 - y3*y4 + y1*y6 + y4*y6 + y0*y7 + y3*y7 + y11 + y12, y7^3 - y7, y4^3 - y4, y8^3 - y8, y10^3 - y10, y12^3 - y12, y0 + y3 - y9, y1 + y4 - y10, -y3 + y6 - y11, -y1*y2 + y2*y4 + y1*y5 + y4*y5 - y9 - y10, -y0*y3 - y3^2 - y0*y6 - y3*y6 - y11, -y1*y4 - y4^2 - y1*y7 - y4*y7 - y12, -y4 + y7 - y12, -y5 + y8 - y12, -y1^2 - y1*y4 + y4^2 - y10, -y0*y2 + y2*y3 + y0*y5 + y3*y5 + y10, y3^3 - y3, y11 + y12 + 1]


In [351]:
set_verbose(0)
I = R2.ideal(total_system)
# J = R2.ideal(flat_system)
print(I.dimension())
J = I.groebner_basis()
print(J)
# Z = I.variety()
# timeit.eval("Z = I.variety()")
# print(Z)
for i in I.gens():
    print(i)

0
[y0 + 1, y1 + 1, y2, y3, y4, y5 + 1, y6 + 1, y7, y8 + 1, y9 + 1, y10 + 1, y11 + 1, y12]
y1^3 - y1
-y2^2 - y2*y5 + y5^2 + y9
y9 + y10 - 1
y2 + y5 - y10
y0^3 - y0
y2^3 - y2
-y0*y1 + y1*y3 + y0*y4 + y3*y4 + y9 + y10
-y2*y5 - y5^2 - y2*y8 - y5*y8 + y11
y11^3 - y11
y2*y4 + y1*y5 - y4*y5 + y2*y7 + y5*y7 + y1*y8 + y4*y8 - y11 - y12
y5^3 - y5
y6^3 - y6
-y0^2 - y0*y3 + y3^2 - y9
y2*y3 + y0*y5 - y3*y5 + y2*y6 + y5*y6 + y0*y8 + y3*y8 + y12
y9^3 - y9
y1*y3 + y0*y4 - y3*y4 + y1*y6 + y4*y6 + y0*y7 + y3*y7 + y11 + y12
y7^3 - y7
y4^3 - y4
y8^3 - y8
y10^3 - y10
y12^3 - y12
y0 + y3 - y9
y1 + y4 - y10
-y3 + y6 - y11
-y1*y2 + y2*y4 + y1*y5 + y4*y5 - y9 - y10
-y0*y3 - y3^2 - y0*y6 - y3*y6 - y11
-y1*y4 - y4^2 - y1*y7 - y4*y7 - y12
-y4 + y7 - y12
-y5 + y8 - y12
-y1^2 - y1*y4 + y4^2 - y10
-y0*y2 + y2*y3 + y0*y5 + y3*y5 + y10
y3^3 - y3
y11 + y12 + 1


In [ ]:
J.ideal().subs(y0=1, y1=2, y12=2)

In [ ]:
L = [f.subs(v) for f in I.gens() for v in Z]
all(f.subs(v) == 0 for f in I.gens() for v in Z)

In [ ]:
# from sage.rings.polynomial.msolve import groebner_basis_degrevlex
# help(variety)
# Z = variety(I, K, proof=False)
# G = groebner_basis_degrevlex(I)
# sorted(variety(I, K, proof=False), key=str)
# print(Z)


In [239]:
# given a system of polynomials, check if it correctly corresponds
# to the actual matrices it was created from
#
# ideal is an ideal of multivariate polynomial ring defined above
# S and T are matrices
def is_correct(ideal, S, T):
    point = S.list() + T.list()

    R = ideal.ring()
    gens = R.gens()
    # make an appropriate dict
    substitution_dict = {gens[i]: point[i] for i in range(len(point))}
    
    return all(f.subs(substitution_dict) == 0 for f in ideal.gens())

In [346]:
is_correct(I, S, T.inverse())

True